# How to use an Encoder-Decoder LSTM to Echo Sequences of Random Integers
https://machinelearningmastery.com/how-to-use-an-encoder-decoder-lstm-to-echo-sequences-of-random-integers/

In [30]:
import numpy as np
import random
import pandas
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, RepeatVector

In [2]:
random.randint(0, 99)

17

In [3]:
# generate a sequence of random numbers in [0, 99]
def generate_sequence(length=25):
    return [random.randint(0, 99) for _ in range(length)]

In [4]:
# one hot encode sequence
def one_hot_encode(sequence, n_unique=100):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return np.array(encoding)

In [5]:
a = one_hot_encode([1,2,3], 5)
a

array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0]])

In [6]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [np.argmax(vector) for vector in encoded_seq]

In [7]:
one_hot_decode(a)

[1, 2, 3]

In [8]:
# convert encoded sequence to supervised learning
def to_supervised(sequence, n_in, n_out):
    # create lag copies of the sequence
    df = DataFrame(sequence)
    df = pandas.concat([df.shift(n_in-i-1) for i in range(n_in)], axis=1)
    # drop rows with missing values
    df.dropna(inplace=True)
    # specify columns for input and output pairs
    values = df.values
    width = sequence.shape[1]
    X = values.reshape(len(values), n_in, width)
    y = values[:, 0:(n_out*width)].reshape(len(values), n_out, width)
    return X, y

In [9]:
sequence = generate_sequence()
print(sequence)

[61, 78, 89, 88, 59, 48, 61, 59, 35, 66, 71, 42, 59, 88, 15, 64, 77, 1, 95, 21, 28, 25, 78, 87, 57]


In [10]:
encoded = one_hot_encode(sequence)

In [11]:
X, y = to_supervised(encoded, 5, 3)

In [12]:
for i in range(len(X)):
    print(one_hot_decode(X[i]), '=>', one_hot_decode(y[i]))

[61, 78, 89, 88, 59] => [61, 78, 89]
[78, 89, 88, 59, 48] => [78, 89, 88]
[89, 88, 59, 48, 61] => [89, 88, 59]
[88, 59, 48, 61, 59] => [88, 59, 48]
[59, 48, 61, 59, 35] => [59, 48, 61]
[48, 61, 59, 35, 66] => [48, 61, 59]
[61, 59, 35, 66, 71] => [61, 59, 35]
[59, 35, 66, 71, 42] => [59, 35, 66]
[35, 66, 71, 42, 59] => [35, 66, 71]
[66, 71, 42, 59, 88] => [66, 71, 42]
[71, 42, 59, 88, 15] => [71, 42, 59]
[42, 59, 88, 15, 64] => [42, 59, 88]
[59, 88, 15, 64, 77] => [59, 88, 15]
[88, 15, 64, 77, 1] => [88, 15, 64]
[15, 64, 77, 1, 95] => [15, 64, 77]
[64, 77, 1, 95, 21] => [64, 77, 1]
[77, 1, 95, 21, 28] => [77, 1, 95]
[1, 95, 21, 28, 25] => [1, 95, 21]
[95, 21, 28, 25, 78] => [95, 21, 28]
[21, 28, 25, 78, 87] => [21, 28, 25]
[28, 25, 78, 87, 57] => [28, 25, 78]


## Echo Whole Sequence (sequence-to-sequence model)

- 入力と出力のシーケンスの長さが同じ場合
- 各タイムステップの入力に対して出力が1つ対応する
- many-to-manyのLSTM

In [13]:
# prepare data for the LSTM
def get_data(n_in, n_out):
    # generate random sequence
    sequence = generate_sequence()
    # one hot encode
    encoded = one_hot_encode(sequence)
    # convert to X,y pairs
    X,y = to_supervised(encoded, n_in, n_out)
    return X,y

In [14]:
model = Sequential()
# (samples, timesteps, features)
model.add(LSTM(20, batch_input_shape=(7, 5, 100), return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(100, activation='softmax')))
#model.add(Dense(100, activation='softmax'))

- `return_sequences=True` なので各タイムステップにおいてベクトルを出力する
- `stateful=True` なのでバッチ間で独立ではなく、前のバッチの最終状態を次のバッチが引き継ぐ
- `stateful=True` の場合は `batch_input_shape` の指定が必要
- `TimeDistributed` にすると各タイムステップの出力に対して `Dense` が適用される

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (7, 5, 20)                9680      
_________________________________________________________________
time_distributed_1 (TimeDist (7, 5, 100)               2100      
Total params: 11,780
Trainable params: 11,780
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [17]:
X,y = get_data(5, 5)

In [18]:
X.shape, y.shape

((21, 5, 100), (21, 5, 100))

In [19]:
# train LSTM
# 各エポックで新規データの生成、内部状態のリセットをするために明示的にepochループが必要
# fit()ではepochs=1にしてバッチの学習のみ
for epoch in range(500):
    # 各エポックで新規データを生成
    X,y = get_data(5, 5)
    # バッチ内では前のバッチの内部状態を引き継いで学習を進める
    # fit model for one epoch on this sequence
    model.fit(X, y, epochs=1, batch_size=7, verbose=2, shuffle=False)
    # エポックの最後で内部状態をリセット
    model.reset_states()

Epoch 1/1
0s - loss: 4.6051 - acc: 0.0476
Epoch 1/1
0s - loss: 4.6067 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.6084 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.6036 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.5999 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.6098 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.6050 - acc: 0.0095
Epoch 1/1
0s - loss: 4.5964 - acc: 0.0571
Epoch 1/1
0s - loss: 4.6011 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.6006 - acc: 0.0190
Epoch 1/1
0s - loss: 4.6014 - acc: 0.0667
Epoch 1/1
0s - loss: 4.5912 - acc: 0.0571
Epoch 1/1
0s - loss: 4.5945 - acc: 0.0190
Epoch 1/1
0s - loss: 4.6072 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.5972 - acc: 0.0095
Epoch 1/1
0s - loss: 4.5960 - acc: 0.0667
Epoch 1/1
0s - loss: 4.5985 - acc: 0.0000e+00
Epoch 1/1
0s - loss: 4.5960 - acc: 0.0476
Epoch 1/1
0s - loss: 4.5827 - acc: 0.0762
Epoch 1/1
0s - loss: 4.5957 - acc: 0.0571
Epoch 1/1
0s - loss: 4.5908 - acc: 0.0095
Epoch 1/1
0s - loss: 4.5871 - acc: 0.0476
Epoch 1/1
0s - loss: 4.5913 - acc: 0.1238
Ep

0s - loss: 3.9192 - acc: 0.5429
Epoch 1/1
0s - loss: 4.0327 - acc: 0.4190
Epoch 1/1
0s - loss: 3.8716 - acc: 0.6667
Epoch 1/1
0s - loss: 4.0738 - acc: 0.4286
Epoch 1/1
0s - loss: 4.0238 - acc: 0.4190
Epoch 1/1
0s - loss: 3.8602 - acc: 0.5333
Epoch 1/1
0s - loss: 3.9079 - acc: 0.6000
Epoch 1/1
0s - loss: 3.8274 - acc: 0.6762
Epoch 1/1
0s - loss: 3.8673 - acc: 0.6190
Epoch 1/1
0s - loss: 4.0154 - acc: 0.4667
Epoch 1/1
0s - loss: 3.8968 - acc: 0.6571
Epoch 1/1
0s - loss: 3.8531 - acc: 0.6762
Epoch 1/1
0s - loss: 3.7839 - acc: 0.6571
Epoch 1/1
0s - loss: 3.8674 - acc: 0.5905
Epoch 1/1
0s - loss: 3.7649 - acc: 0.6762
Epoch 1/1
0s - loss: 3.7625 - acc: 0.6857
Epoch 1/1
0s - loss: 3.8752 - acc: 0.5524
Epoch 1/1
0s - loss: 3.9389 - acc: 0.5048
Epoch 1/1
0s - loss: 3.8629 - acc: 0.6095
Epoch 1/1
0s - loss: 3.7637 - acc: 0.6476
Epoch 1/1
0s - loss: 3.7844 - acc: 0.6857
Epoch 1/1
0s - loss: 3.7842 - acc: 0.5714
Epoch 1/1
0s - loss: 3.8328 - acc: 0.6286
Epoch 1/1
0s - loss: 3.8422 - acc: 0.5238
Ep

0s - loss: 2.4695 - acc: 1.0000
Epoch 1/1
0s - loss: 2.4774 - acc: 0.9333
Epoch 1/1
0s - loss: 2.4871 - acc: 0.9810
Epoch 1/1
0s - loss: 2.3252 - acc: 1.0000
Epoch 1/1
0s - loss: 2.1293 - acc: 1.0000
Epoch 1/1
0s - loss: 2.3753 - acc: 1.0000
Epoch 1/1
0s - loss: 2.4328 - acc: 0.9714
Epoch 1/1
0s - loss: 2.3464 - acc: 1.0000
Epoch 1/1
0s - loss: 2.4524 - acc: 0.9524
Epoch 1/1
0s - loss: 2.4050 - acc: 0.9619
Epoch 1/1
0s - loss: 2.2978 - acc: 0.9905
Epoch 1/1
0s - loss: 2.5053 - acc: 0.9524
Epoch 1/1
0s - loss: 2.2653 - acc: 0.9905
Epoch 1/1
0s - loss: 2.2931 - acc: 1.0000
Epoch 1/1
0s - loss: 2.3981 - acc: 1.0000
Epoch 1/1
0s - loss: 2.3575 - acc: 0.9238
Epoch 1/1
0s - loss: 2.3438 - acc: 0.9429
Epoch 1/1
0s - loss: 2.1342 - acc: 0.9619
Epoch 1/1
0s - loss: 2.1598 - acc: 0.9714
Epoch 1/1
0s - loss: 2.4522 - acc: 0.9524
Epoch 1/1
0s - loss: 2.2487 - acc: 1.0000
Epoch 1/1
0s - loss: 2.0333 - acc: 1.0000
Epoch 1/1
0s - loss: 1.9698 - acc: 1.0000
Epoch 1/1
0s - loss: 2.2631 - acc: 0.9524
Ep

In [20]:
# evaluate LSTM
X,y = get_data(5, 5)
yhat = model.predict(X, batch_size=7, verbose=0)
# decode all pairs
for i in range(len(X)):
    print('Expected:', one_hot_decode(y[i]), 'Predicted', one_hot_decode(yhat[i]))

Expected: [15, 31, 20, 38, 61] Predicted [15, 31, 20, 38, 61]
Expected: [31, 20, 38, 61, 91] Predicted [31, 20, 38, 61, 91]
Expected: [20, 38, 61, 91, 93] Predicted [20, 38, 61, 91, 93]
Expected: [38, 61, 91, 93, 99] Predicted [38, 61, 91, 93, 99]
Expected: [61, 91, 93, 99, 83] Predicted [61, 91, 93, 99, 83]
Expected: [91, 93, 99, 83, 52] Predicted [91, 93, 99, 83, 52]
Expected: [93, 99, 83, 52, 30] Predicted [93, 99, 83, 52, 30]
Expected: [99, 83, 52, 30, 84] Predicted [99, 83, 52, 30, 84]
Expected: [83, 52, 30, 84, 66] Predicted [83, 52, 30, 84, 66]
Expected: [52, 30, 84, 66, 85] Predicted [52, 30, 84, 66, 85]
Expected: [30, 84, 66, 85, 46] Predicted [30, 84, 66, 85, 46]
Expected: [84, 66, 85, 46, 20] Predicted [84, 66, 85, 46, 20]
Expected: [66, 85, 46, 20, 71] Predicted [66, 85, 46, 20, 71]
Expected: [85, 46, 20, 71, 78] Predicted [85, 46, 20, 71, 78]
Expected: [46, 20, 71, 78, 55] Predicted [46, 20, 71, 78, 55]
Expected: [20, 71, 78, 55, 20] Predicted [20, 71, 78, 55, 20]
Expected

## Echo Partial Sequence (encoder-decoder model)

In [41]:
model = Sequential()
# return_sequences=Falseなので最後の入力に対する出力のみ encoderの出力（fixed context vector）は (21, 150)
model.add(LSTM(150, batch_input_shape=(21, 5, 100), stateful=True))
# decoderの出力は (21, ?, 150) であってほしい ?はdecoderの系列長
# encoderの出力が繰り返される（context vectorが各ステップに入力される
# decoderの系列長は RepeatVector層を加える 2 timesteps
model.add(RepeatVector(2))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(100, activation='softmax')))

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (21, 150)                 150600    
_________________________________________________________________
repeat_vector_6 (RepeatVecto (21, 2, 150)              0         
_________________________________________________________________
lstm_19 (LSTM)               (21, 2, 150)              180600    
_________________________________________________________________
time_distributed_2 (TimeDist (21, 2, 100)              15100     
Total params: 346,300
Trainable params: 346,300
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [47]:
n_in = 5
n_out = 2
encoded_length = 100
batch_size = 21

model = Sequential()
model.add(LSTM(150, batch_input_shape=(batch_size, n_in, encoded_length), stateful=True))
model.add(RepeatVector(n_out))
model.add(LSTM(150, return_sequences=True, stateful=True))
model.add(TimeDistributed(Dense(encoded_length, activation='softmax')))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam')

for epoch in range(5000):
    X, y = get_data(n_in, n_out)
    model.fit(X, y, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (21, 150)                 150600    
_________________________________________________________________
repeat_vector_9 (RepeatVecto (21, 2, 150)              0         
_________________________________________________________________
lstm_25 (LSTM)               (21, 2, 150)              180600    
_________________________________________________________________
time_distributed_5 (TimeDist (21, 2, 100)              15100     
Total params: 346,300
Trainable params: 346,300
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1
0s - loss: 4.6047
Epoch 1/1
0s - loss: 4.6053
Epoch 1/1
0s - loss: 4.6058
Epoch 1/1
0s - loss: 4.6031
Epoch 1/1
0s - loss: 4.6039
Epoch 1/1
0s - loss: 4.6044
Epoch 1/1
0s - loss: 4.6044
Epoch 1/1
0s - loss: 4.6027
Epoch 1/1
0s - loss: 4.6081
Epoch 1/1
0s - l

0s - loss: 2.4426
Epoch 1/1
0s - loss: 2.4069
Epoch 1/1
0s - loss: 2.2959
Epoch 1/1
0s - loss: 2.5175
Epoch 1/1
0s - loss: 2.6718
Epoch 1/1
0s - loss: 2.3523
Epoch 1/1
0s - loss: 2.4013
Epoch 1/1
0s - loss: 2.7190
Epoch 1/1
0s - loss: 2.4633
Epoch 1/1
0s - loss: 2.4678
Epoch 1/1
0s - loss: 2.4087
Epoch 1/1
0s - loss: 2.4614
Epoch 1/1
0s - loss: 2.5606
Epoch 1/1
0s - loss: 2.8792
Epoch 1/1
0s - loss: 2.4886
Epoch 1/1
0s - loss: 2.5639
Epoch 1/1
0s - loss: 2.3896
Epoch 1/1
0s - loss: 2.4261
Epoch 1/1
0s - loss: 2.6137
Epoch 1/1
0s - loss: 2.3710
Epoch 1/1
0s - loss: 2.5045
Epoch 1/1
0s - loss: 2.4857
Epoch 1/1
0s - loss: 2.4649
Epoch 1/1
0s - loss: 2.5187
Epoch 1/1
0s - loss: 2.5368
Epoch 1/1
0s - loss: 2.8489
Epoch 1/1
0s - loss: 2.6816
Epoch 1/1
0s - loss: 2.1892
Epoch 1/1
0s - loss: 2.4737
Epoch 1/1
0s - loss: 2.4553
Epoch 1/1
0s - loss: 2.4367
Epoch 1/1
0s - loss: 2.1217
Epoch 1/1
0s - loss: 2.2106
Epoch 1/1
0s - loss: 2.4465
Epoch 1/1
0s - loss: 2.4063
Epoch 1/1
0s - loss: 2.5168
Ep

Epoch 1/1
0s - loss: 1.8049
Epoch 1/1
0s - loss: 1.8272
Epoch 1/1
0s - loss: 1.6852
Epoch 1/1
0s - loss: 1.6820
Epoch 1/1
0s - loss: 1.6480
Epoch 1/1
0s - loss: 1.5865
Epoch 1/1
0s - loss: 1.6787
Epoch 1/1
0s - loss: 1.7957
Epoch 1/1
0s - loss: 1.7906
Epoch 1/1
0s - loss: 1.8546
Epoch 1/1
0s - loss: 1.8515
Epoch 1/1
0s - loss: 1.9734
Epoch 1/1
0s - loss: 1.8025
Epoch 1/1
0s - loss: 1.9132
Epoch 1/1
0s - loss: 1.6235
Epoch 1/1
0s - loss: 1.7252
Epoch 1/1
0s - loss: 1.7122
Epoch 1/1
0s - loss: 1.8775
Epoch 1/1
0s - loss: 1.8182
Epoch 1/1
0s - loss: 1.8283
Epoch 1/1
0s - loss: 1.7361
Epoch 1/1
0s - loss: 1.6770
Epoch 1/1
0s - loss: 1.8994
Epoch 1/1
0s - loss: 1.9036
Epoch 1/1
0s - loss: 1.7240
Epoch 1/1
0s - loss: 1.7159
Epoch 1/1
0s - loss: 1.6041
Epoch 1/1
0s - loss: 1.5706
Epoch 1/1
0s - loss: 1.5954
Epoch 1/1
0s - loss: 1.8679
Epoch 1/1
0s - loss: 1.6388
Epoch 1/1
0s - loss: 1.7030
Epoch 1/1
0s - loss: 1.6696
Epoch 1/1
0s - loss: 1.6947
Epoch 1/1
0s - loss: 1.5084
Epoch 1/1
0s - loss:

Epoch 1/1
0s - loss: 1.2860
Epoch 1/1
0s - loss: 1.4291
Epoch 1/1
0s - loss: 1.4032
Epoch 1/1
0s - loss: 1.7807
Epoch 1/1
0s - loss: 1.5611
Epoch 1/1
0s - loss: 1.4023
Epoch 1/1
0s - loss: 1.3278
Epoch 1/1
0s - loss: 1.3147
Epoch 1/1
0s - loss: 1.6315
Epoch 1/1
0s - loss: 1.3189
Epoch 1/1
0s - loss: 1.4367
Epoch 1/1
0s - loss: 1.2804
Epoch 1/1
0s - loss: 1.3234
Epoch 1/1
0s - loss: 1.4773
Epoch 1/1
0s - loss: 1.2254
Epoch 1/1
0s - loss: 1.3555
Epoch 1/1
0s - loss: 1.3429
Epoch 1/1
0s - loss: 1.5195
Epoch 1/1
0s - loss: 1.3787
Epoch 1/1
0s - loss: 1.2976
Epoch 1/1
0s - loss: 1.2940
Epoch 1/1
0s - loss: 1.3147
Epoch 1/1
0s - loss: 1.8014
Epoch 1/1
0s - loss: 1.4199
Epoch 1/1
0s - loss: 1.5264
Epoch 1/1
0s - loss: 1.4989
Epoch 1/1
0s - loss: 1.5207
Epoch 1/1
0s - loss: 1.5827
Epoch 1/1
0s - loss: 1.3367
Epoch 1/1
0s - loss: 1.5876
Epoch 1/1
0s - loss: 1.3651
Epoch 1/1
0s - loss: 1.4917
Epoch 1/1
0s - loss: 1.4408
Epoch 1/1
0s - loss: 1.5100
Epoch 1/1
0s - loss: 1.4767
Epoch 1/1
0s - loss:

0s - loss: 1.2346
Epoch 1/1
0s - loss: 1.3555
Epoch 1/1
0s - loss: 1.1917
Epoch 1/1
0s - loss: 1.1720
Epoch 1/1
0s - loss: 1.1931
Epoch 1/1
0s - loss: 1.1667
Epoch 1/1
0s - loss: 1.1800
Epoch 1/1
0s - loss: 1.0832
Epoch 1/1
0s - loss: 0.9648
Epoch 1/1
0s - loss: 1.2026
Epoch 1/1
0s - loss: 1.2160
Epoch 1/1
0s - loss: 1.1413
Epoch 1/1
0s - loss: 1.1518
Epoch 1/1
0s - loss: 1.1155
Epoch 1/1
0s - loss: 1.1297
Epoch 1/1
0s - loss: 1.1497
Epoch 1/1
0s - loss: 1.2594
Epoch 1/1
0s - loss: 1.0852
Epoch 1/1
0s - loss: 1.0647
Epoch 1/1
0s - loss: 1.0378
Epoch 1/1
0s - loss: 1.0389
Epoch 1/1
0s - loss: 1.1135
Epoch 1/1
0s - loss: 1.2363
Epoch 1/1
0s - loss: 1.0365
Epoch 1/1
0s - loss: 1.0966
Epoch 1/1
0s - loss: 1.2689
Epoch 1/1
0s - loss: 1.0948
Epoch 1/1
0s - loss: 1.0217
Epoch 1/1
0s - loss: 1.0636
Epoch 1/1
0s - loss: 1.0440
Epoch 1/1
0s - loss: 1.0692
Epoch 1/1
0s - loss: 1.0857
Epoch 1/1
0s - loss: 1.1001
Epoch 1/1
0s - loss: 1.0812
Epoch 1/1
0s - loss: 1.1001
Epoch 1/1
0s - loss: 0.9676
Ep

...

0s - loss: 0.0692
Epoch 1/1
0s - loss: 0.0429
Epoch 1/1
0s - loss: 0.1624
Epoch 1/1
0s - loss: 0.0705
Epoch 1/1
0s - loss: 0.1083
Epoch 1/1
0s - loss: 0.0627
Epoch 1/1
0s - loss: 0.1310
Epoch 1/1
0s - loss: 0.0839
Epoch 1/1
0s - loss: 0.0657
Epoch 1/1
0s - loss: 0.0935
Epoch 1/1
0s - loss: 0.0786
Epoch 1/1
0s - loss: 0.0803
Epoch 1/1
0s - loss: 0.0619
Epoch 1/1
0s - loss: 0.1666
Epoch 1/1
0s - loss: 0.0746
Epoch 1/1
0s - loss: 0.1146
Epoch 1/1
0s - loss: 0.0908
Epoch 1/1
0s - loss: 0.0682
Epoch 1/1
0s - loss: 0.0557
Epoch 1/1
0s - loss: 0.0825
Epoch 1/1
0s - loss: 0.0614
Epoch 1/1
0s - loss: 0.0629
Epoch 1/1
0s - loss: 0.0615
Epoch 1/1
0s - loss: 0.1124
Epoch 1/1
0s - loss: 0.0522
Epoch 1/1
0s - loss: 0.0862
Epoch 1/1
0s - loss: 0.0797
Epoch 1/1
0s - loss: 0.0408
Epoch 1/1
0s - loss: 0.0825
Epoch 1/1
0s - loss: 0.0520
Epoch 1/1
0s - loss: 0.0686
Epoch 1/1
0s - loss: 0.0989
Epoch 1/1
0s - loss: 0.0773
Epoch 1/1
0s - loss: 0.0400
Epoch 1/1
0s - loss: 0.0308
Epoch 1/1
0s - loss: 0.0738
Ep

Epoch 1/1
0s - loss: 0.0629
Epoch 1/1
0s - loss: 0.0650
Epoch 1/1
0s - loss: 0.0696
Epoch 1/1
0s - loss: 0.0596
Epoch 1/1
0s - loss: 0.0804
Epoch 1/1
0s - loss: 0.0415
Epoch 1/1
0s - loss: 0.0276
Epoch 1/1
0s - loss: 0.0990
Epoch 1/1
0s - loss: 0.0613
Epoch 1/1
0s - loss: 0.0713
Epoch 1/1
0s - loss: 0.0675
Epoch 1/1
0s - loss: 0.0523
Epoch 1/1
0s - loss: 0.0604
Epoch 1/1
0s - loss: 0.0507
Epoch 1/1
0s - loss: 0.0435
Epoch 1/1
0s - loss: 0.0479
Epoch 1/1
0s - loss: 0.0447
Epoch 1/1
0s - loss: 0.0518
Epoch 1/1
0s - loss: 0.0682
Epoch 1/1
0s - loss: 0.0244
Epoch 1/1
0s - loss: 0.0455
Epoch 1/1
0s - loss: 0.0646
Epoch 1/1
0s - loss: 0.0684
Epoch 1/1
0s - loss: 0.0452
Epoch 1/1
0s - loss: 0.0269
Epoch 1/1
0s - loss: 0.0394
Epoch 1/1
0s - loss: 0.1072
Epoch 1/1
0s - loss: 0.0714
Epoch 1/1
0s - loss: 0.0607
Epoch 1/1
0s - loss: 0.0553
Epoch 1/1
0s - loss: 0.0377
Epoch 1/1
0s - loss: 0.0990
Epoch 1/1
0s - loss: 0.0597
Epoch 1/1
0s - loss: 0.0482
Epoch 1/1
0s - loss: 0.0593
Epoch 1/1
0s - loss:

Epoch 1/1
0s - loss: 0.0482
Epoch 1/1
0s - loss: 0.0466
Epoch 1/1
0s - loss: 0.0581
Epoch 1/1
0s - loss: 0.0490
Epoch 1/1
0s - loss: 0.0381
Epoch 1/1
0s - loss: 0.0522
Epoch 1/1
0s - loss: 0.0658
Epoch 1/1
0s - loss: 0.0557
Epoch 1/1
0s - loss: 0.1159
Epoch 1/1
0s - loss: 0.0485
Epoch 1/1
0s - loss: 0.0708
Epoch 1/1
0s - loss: 0.0722
Epoch 1/1
0s - loss: 0.0376
Epoch 1/1
0s - loss: 0.0429
Epoch 1/1
0s - loss: 0.0371
Epoch 1/1
0s - loss: 0.0483
Epoch 1/1
0s - loss: 0.0647
Epoch 1/1
0s - loss: 0.0780
Epoch 1/1
0s - loss: 0.0394
Epoch 1/1
0s - loss: 0.0347
Epoch 1/1
0s - loss: 0.0389
Epoch 1/1
0s - loss: 0.0408
Epoch 1/1
0s - loss: 0.0786
Epoch 1/1
0s - loss: 0.0395
Epoch 1/1
0s - loss: 0.1060
Epoch 1/1
0s - loss: 0.0569
Epoch 1/1
0s - loss: 0.0377
Epoch 1/1
0s - loss: 0.0473
Epoch 1/1
0s - loss: 0.0271
Epoch 1/1
0s - loss: 0.0343
Epoch 1/1
0s - loss: 0.0347
Epoch 1/1
0s - loss: 0.0528
Epoch 1/1
0s - loss: 0.0345
Epoch 1/1
0s - loss: 0.0197
Epoch 1/1
0s - loss: 0.0377
Epoch 1/1
0s - loss:

0s - loss: 0.0572
Epoch 1/1
0s - loss: 0.0371
Epoch 1/1
0s - loss: 0.0725
Epoch 1/1
0s - loss: 0.0349
Epoch 1/1
0s - loss: 0.0411
Epoch 1/1
0s - loss: 0.0633
Epoch 1/1
0s - loss: 0.0267
Epoch 1/1
0s - loss: 0.0421
Epoch 1/1
0s - loss: 0.0233
Epoch 1/1
0s - loss: 0.0383
Epoch 1/1
0s - loss: 0.0289
Epoch 1/1
0s - loss: 0.0408
Epoch 1/1
0s - loss: 0.0427
Epoch 1/1
0s - loss: 0.0281
Epoch 1/1
0s - loss: 0.0825
Epoch 1/1
0s - loss: 0.0291
Epoch 1/1
0s - loss: 0.0374
Epoch 1/1
0s - loss: 0.0616
Epoch 1/1
0s - loss: 0.0349
Epoch 1/1
0s - loss: 0.0287
Epoch 1/1
0s - loss: 0.0690
Epoch 1/1
0s - loss: 0.0831
Epoch 1/1
0s - loss: 0.0377
Epoch 1/1
0s - loss: 0.0574
Epoch 1/1
0s - loss: 0.0503
Epoch 1/1
0s - loss: 0.0264
Epoch 1/1
0s - loss: 0.0358
Epoch 1/1
0s - loss: 0.0384
Epoch 1/1
0s - loss: 0.0416
Epoch 1/1
0s - loss: 0.0207
Epoch 1/1
0s - loss: 0.0249
Epoch 1/1
0s - loss: 0.0491
Epoch 1/1
0s - loss: 0.0529
Epoch 1/1
0s - loss: 0.0365
Epoch 1/1
0s - loss: 0.0404
Epoch 1/1
0s - loss: 0.0658
Ep

0s - loss: 0.0735
Epoch 1/1
0s - loss: 0.0446
Epoch 1/1
0s - loss: 0.0301
Epoch 1/1
0s - loss: 0.0299


In [48]:
# evaluate LSTM
X, y = get_data(n_in, n_out)
yhat = model.predict(X, batch_size=batch_size, verbose=0)
for i in range(len(X)):
    print('expected:', one_hot_decode(y[i]), 'predicted:', one_hot_decode(yhat[i]))

expected: [53, 27] predicted: [53, 27]
expected: [27, 15] predicted: [27, 15]
expected: [15, 26] predicted: [15, 26]
expected: [26, 85] predicted: [26, 85]
expected: [85, 96] predicted: [85, 96]
expected: [96, 35] predicted: [96, 35]
expected: [35, 19] predicted: [35, 19]
expected: [19, 81] predicted: [19, 81]
expected: [81, 23] predicted: [81, 23]
expected: [23, 59] predicted: [23, 59]
expected: [59, 63] predicted: [59, 63]
expected: [63, 23] predicted: [63, 23]
expected: [23, 81] predicted: [23, 81]
expected: [81, 15] predicted: [81, 15]
expected: [15, 60] predicted: [15, 60]
expected: [60, 83] predicted: [60, 83]
expected: [83, 41] predicted: [83, 41]
expected: [41, 94] predicted: [41, 94]
expected: [94, 0] predicted: [94, 0]
expected: [0, 88] predicted: [0, 88]
expected: [88, 70] predicted: [88, 70]
